# Overview
This is a take on the Assistants API version for [Movies Assistant](movie-assistant.ipynb) using the new Azure AI Agent Service. The service is structured like the Assistants API so one can easily convert the code

Follow the steps to create the movies.csv:
1. Open up the `Chat` interface in https://oai.azure.com and use the following
    - Prompt: Generate me a list ficticious movies with a name column and a rating column. Output in CSV format
1. Paste the output into the `movies.csv` file

Here is my config for my conda env:
```bash
conda create -n agent python=3.13
conda activate agent
pip install azure-ai-projects
pip install azure-identity
pip install ipykernel
pip install python-dotenv
```

What you need in the `.env` file:
```text
PROJECT_CONNECTION_STRING=<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>
```

You can get this directly off the Overview of an Azure AI Project

## References
- https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart?pivots=programming-language-python-azure
- https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects
    - A ton of examples


In [5]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FilePurpose
from azure.ai.projects.models import CodeInterpreterTool
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential

# Load the .env file into the environment
_ = load_dotenv()

In [3]:
conn_str=os.environ["PROJECT_CONNECTION_STRING"]

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=conn_str,
)

In [6]:
# Upload the file
file = project_client.agents.upload_file_and_poll(
    file_path='movies.csv', purpose=FilePurpose.AGENTS
)
print(f"Uploaded file, file ID: {file.id}")

Uploaded file, file ID: assistant-tFEsQscA4TfJWh91kba3PI8u


In [7]:
# Create an instance of the CodeInterpreterTool
code_interpreter = CodeInterpreterTool(file_ids=[file.id])

# The CodeInterpreterTool needs to be included in creation of the agent
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="movie agent",
    instructions="You are a movie analyst. When asked a question, you will parse your CSV file to provide the requested analysis.",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_sh4SHLlw9YXwitx99ZsFWYJZ


In [10]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_TbW7ErIzTw9p8vTvCawgqhd7


In [8]:
def ask_a_question(question: str):
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question,
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")


In [11]:
ask_a_question("What was the highest rated movie?")


Created message, message ID: msg_wlZb3PDMIg7E6OVxkX5QL0q4
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_Cbv5SMtqBZrsk9JfG2Jj8K3W', 'object': 'thread.message', 'created_at': 1734618771, 'assistant_id': 'asst_sh4SHLlw9YXwitx99ZsFWYJZ', 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': 'run_4Sv8PiIbqA45xMjZzuyz4MXH', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The highest rated movie is **"Shadows of Tomorrow"** with a rating of **9.1**.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_wlZb3PDMIg7E6OVxkX5QL0q4', 'object': 'thread.message', 'created_at': 1734618762, 'assistant_id': None, 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What was the highest rated movie?', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_Cbv5SMtqBZrsk9JfG2Jj8K3W', 'last_id': 'msg_wlZb3PDMIg7E6OVxkX5Q

In [12]:
ask_a_question("What are the top 5 movies by rating?")

Created message, message ID: msg_xlidALdiFhWungXpF2Th9jX3
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_ArkrMiXROlV9xoZTRbR8lrad', 'object': 'thread.message', 'created_at': 1734618802, 'assistant_id': 'asst_sh4SHLlw9YXwitx99ZsFWYJZ', 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': 'run_oi8SkPevmP3yvRQGxtFTFONW', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The top 5 movies by rating are:\n\n1. **Shadows of Tomorrow** - Rating: 9.1\n2. **Starlight Beyond Time** - Rating: 9.0\n3. **Secrets of the Alabaster Tower** - Rating: 8.9\n4. **The Whispering Echoes** - Rating: 8.7\n5. **Voyage through the Hidden Isles** - Rating: 8.6', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_xlidALdiFhWungXpF2Th9jX3', 'object': 'thread.message', 'created_at': 1734618799, 'assistant_id': None, 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 't

In [13]:
ask_a_question("What is the average rating of the movies?")


Created message, message ID: msg_zmThxLzaqkv6GZ37YjiYBxe3
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_opKeBk2keMonP2H9rkqULApc', 'object': 'thread.message', 'created_at': 1734618837, 'assistant_id': 'asst_sh4SHLlw9YXwitx99ZsFWYJZ', 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': 'run_jYeUw9OrFiw6FFoDEuzvxZuK', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The average rating of the movies is approximately **7.99**.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_zmThxLzaqkv6GZ37YjiYBxe3', 'object': 'thread.message', 'created_at': 1734618831, 'assistant_id': None, 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is the average rating of the movies?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_ArkrMiXROlV9xoZTRbR8lrad', 'object': 'thread.message', 'created_at': 1734618802,

In [14]:
ask_a_question("What is the lowest rate movies?")


Created message, message ID: msg_yeGXIx8cUJjtFPJLVFG0SeY1
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_67o2adUYcuj2q89xvV0HhwKM', 'object': 'thread.message', 'created_at': 1734618867, 'assistant_id': 'asst_sh4SHLlw9YXwitx99ZsFWYJZ', 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': 'run_jEZvjYMyKUd2KE1qwe5YjTGm', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The lowest rated movie is **"Echoes in the Silence"** with a rating of **6.8**.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_yeGXIx8cUJjtFPJLVFG0SeY1', 'object': 'thread.message', 'created_at': 1734618864, 'assistant_id': None, 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is the lowest rate movies?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_opKeBk2keMonP2H9rkqULApc', 'object': 'thread.message', 'created_at': 1

In [15]:
ask_a_question("How many movies are there?")


Created message, message ID: msg_nszqPeaFOCNCIt821QdIOSmi
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_0gLdrCgo8ZylqxDLtXXCdEnB', 'object': 'thread.message', 'created_at': 1734618881, 'assistant_id': 'asst_sh4SHLlw9YXwitx99ZsFWYJZ', 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': 'run_o5dARDDovQsnqXA73TImnrfb', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'There are a total of **20 movies** in the dataset.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_nszqPeaFOCNCIt821QdIOSmi', 'object': 'thread.message', 'created_at': 1734618878, 'assistant_id': None, 'thread_id': 'thread_TbW7ErIzTw9p8vTvCawgqhd7', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'How many movies are there?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_67o2adUYcuj2q89xvV0HhwKM', 'object': 'thread.message', 'created_at': 1734618867, 'assistant_id': 'asst_s

# Clean Up

In [16]:
project_client.agents.delete_file(file.id)
print("Deleted file")

project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted file
Deleted agent
